### Oracle - cx_Oracle

#### 1. Connecting to Oracle

In [1]:
!pip install cx_oracle

In [2]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle

# 오라클 서비스 시작 후 실행
con = cx_Oracle.connect(user='scott', password='tiger', dsn='localhost:1522/orcl')

cur = con.cursor()   # SQL 수행 결과를 담을 메모리 공간
cur.execute("""
            SELECT *
            FROM DEPT
        """)
res = cur.fetchall()  # 커서에 담긴 내용을 모두 꺼내옴
for row in res:
    print(row)


(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')


#### 2. Connection Pooling

In [3]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


def Query():
    con = pool.acquire()
    cur = con.cursor()
    for i in range(4):
        cur.execute(f"""
                    SELECT {i}
                      FROM DUAL
                """)
                
        seqval, = cur.fetchone()
        print("Thread",
              threading.current_thread().name,
              "fetched sequence =", seqval)


thread1 = threading.Thread(name='#1', target=Query)
thread1.start()

thread2 = threading.Thread(name='#2', target=Query)
thread2.start()

thread1.join()
thread2.join()

print("All done!")


ThreadThread #1 fetched sequence = 0
 #2 fetched sequence = 0
Thread #2 fetched sequence = 1
Thread #2 fetched sequence = 2
Thread #2 fetched sequence = 3
Thread #1 fetched sequence = 1
Thread #1 fetched sequence = 2
Thread #1 fetched sequence = 3
All done!


#### 스레딩 연습

In [4]:
# thread_test.py
import time

def long_task():  # 5초의 시간이 걸리는 함수
    for i in range(5):
        time.sleep(1)  # 1초간 대기한다.
        print("working:%s\n" % i)

print("Start")

for i in range(5):  # long_task를 5회 수행한다.
    long_task()

print("End")

Start
working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

End


In [6]:
import time
import threading  # 스레드를 생성하기 위해서는 threading 모듈이 필요하다.

def long_task():
    for i in range(5):
        time.sleep(1)
        print("working:%s\n" % i)

print("Start")

threads = []
for i in range(5):
    t = threading.Thread(target=long_task)  # 스레드를 생성한다.
    threads.append(t) 

for t in threads:
    t.start()  # 스레드를 실행한다.

for t in threads:
    t.join()  # join으로 스레드가 종료될때까지 기다린다.
    
print("End")

Start
working:0

working:0
working:0


working:0

working:0

working:1

working:1

working:1
working:1


working:1

working:2

working:2

working:2
working:2


working:2

working:3

working:3
working:3

working:3


working:3

working:4

working:4

working:4
working:4


working:4

End


#### 3. Fetching Data

##### 3.1 A simple query

In [7]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

cur.execute("""
            SELECT *
              FROM DEPT
            ORDER BY DEPTNO
        """)

for deptno, dname, loc in cur:
    print("Department number: ", deptno)
    print("Department name: ", dname)
    print("Department location:", loc)
    print()


Department number:  10
Department name:  ACCOUNTING
Department location: NEW YORK

Department number:  20
Department name:  RESEARCH
Department location: DALLAS

Department number:  30
Department name:  SALES
Department location: CHICAGO

Department number:  40
Department name:  OPERATIONS
Department location: BOSTON



##### 3.2 Using fetchone()

In [8]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

cur.execute("""
            SELECT *
              FROM DEPT
            ORDER BY DEPTNO
        """)

row = cur.fetchone()
print(row)

row = cur.fetchone()
print(row)


(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')


##### 3.3 Using fetchmany()

In [10]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

cur.execute("""
            SELECT *
              FROM DEPT
            ORDER BY DEPTNO
        """)

res = cur.fetchmany(numRows = 3)
print(res)
print()
print(res[0])    # first row
print()
print(res[0][1]) # second element of first row


[(10, 'ACCOUNTING', 'NEW YORK'), (20, 'RESEARCH', 'DALLAS'), (30, 'SALES', 'CHICAGO')]

(10, 'ACCOUNTING', 'NEW YORK')

ACCOUNTING


#### 4. Binding Data

##### 4.1 Binding in queries

In [12]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

sql = """
    SELECT *
      FROM DEPT
     WHERE DEPTNO = :id
    ORDER BY DEPTNO
"""

cur.execute(sql, id = 20)
res = cur.fetchall()
print(res)

cur.execute(sql, id = 10)
res = cur.fetchall()
print(res)


[(20, 'RESEARCH', 'DALLAS')]
[(10, 'ACCOUNTING', 'NEW YORK')]


##### 4.2 Binding in inserts

In [13]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

# Create temporary table
cur.execute("DROP TABLE MYTAB")
cur.execute("CREATE TABLE MYTAB (ID NUMBER, DATA VARCHAR2(100) UNIQUE)")

rows = [ (1, "First" ),
         (2, "Second" ),
         (3, "Third" ),
         (4, "Fourth" ),
         (5, "Fifth" ),
         (6, "Sixth" ),
         (7, "Seventh" ) ]

# Insert data
cur.executemany("""
                INSERT INTO MYTAB (id, data)
                           VALUES (:1, :2)
                """
                , rows)

# con.commit()
# con.rollback()

# Now query the results back
cur.execute("SELECT * FROM MYTAB")
res = cur.fetchall()
print(res)


[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (5, 'Fifth'), (6, 'Sixth'), (7, 'Seventh')]


##### 4.3 Batcherrors

In [14]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

# Create temporary table
cur.execute("DROP TABLE MYTAB")
cur.execute("CREATE TABLE MYTAB (ID NUMBER, DATA VARCHAR2(100) UNIQUE)")

rows = [ (1, "First" ), (2, "Second" ),
         (3, "Third" ), (4, "Fourth" ),
         (5, "Third" ), (6, "Sixth" ),
         (7, "Seventh" ) ]

# Insert data
cur.executemany("""
                INSERT INTO MYTAB (id, data)
                           VALUES (:1, :2)
                """
                , rows
                , batcherrors=True)

for error in cur.getbatcherrors():
    print("Error", error.message.rstrip(), "at row offset", error.offset)

# con.commit()
# con.rollback()

# Now query the results back
cur.execute("SELECT * FROM MYTAB")
res = cur.fetchall()
print(res)


Error ORA-00001: 무결성 제약 조건(SCOTT.SYS_C0011161)에 위배됩니다 at row offset 4
[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (6, 'Sixth'), (7, 'Seventh')]


#### DB - Pandas 연동

In [16]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading
import pandas as pd


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

# Create temporary table
cur.execute("DROP TABLE MYTAB")
cur.execute("CREATE TABLE MYTAB (ID NUMBER, DATA VARCHAR2(100) UNIQUE)")

rows = [ (1, "First" ), (2, "Second" ),
         (3, "Third" ), (4, "Fourth" ),
         (5, "Fifth" ), (6, "Sixth" ),
         (7, "Seventh" ) ]

# Insert data
cur.executemany("""
                INSERT INTO MYTAB (id, data)
                           VALUES (:1, :2)
                """
                , rows)

# con.commit()
# con.rollback()

# Now query the results back
cur.execute("SELECT * FROM MYTAB")
res = cur.fetchall()
print(res)

df = pd.read_sql("SELECT * FROM MYTAB", con)
df


[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (5, 'Fifth'), (6, 'Sixth'), (7, 'Seventh')]


,ID,DATA
0,1,First
1,2,Second
2,3,Third
3,4,Fourth
4,5,Fifth
5,6,Sixth
6,7,Seventh


### SQLite

- https://docs.python.org/ko/3/library/sqlite3.html
- https://wikidocs.net/book/1530

#### 1. Connection

In [1]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect('example.db')


#### 2. Data

In [3]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect('example.db')
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE STOCKS
               (DATE TEXT, TRANS TEXT, SYMBOL TEXT, QTY REAL, PRICE REAL)''')

# Insert a row of data
cur.execute("INSERT INTO STOCKS VALUES ('2006-01-05', 'BUY', 'RHAT', 100, 35.14)")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()


OperationalError: table STOCKS already exists

#### 2. Selection

In [4]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect('example.db')
cur = con.cursor()

for row in cur.execute('SELECT * FROM STOCKS ORDER BY PRICE'):
    print(row)


('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


#### 3. Database in Memory

In [8]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("CREATE TABLE LANG (NAME, FIRST_APPEARED)")

# This is the qmark style:
cur.execute("INSERT INTO LANG VALUES (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cur.executemany("INSERT INTO LANG VALUES (?, ?)", lang_list)

# And this is the named style:
cur.execute("SELECT * FROM LANG WHERE FIRST_APPEARED=:year", {"year": 1972})
print(cur.fetchall())

con.close()


[('C', 1972)]


#### DB - Pandas 연동

In [10]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3
import pandas as pd


con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("CREATE TABLE LANG (NAME, FIRST_APPEARED)")

# This is the qmark style:
cur.execute("INSERT INTO LANG VALUES (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cur.executemany("INSERT INTO LANG VALUES (?, ?)", lang_list)

# And this is the named style:
cur.execute("SELECT * FROM LANG WHERE FIRST_APPEARED=:year", {"year": 1972})
rows = cur.fetchall()
print(rows)

df = pd.read_sql("SELECT * FROM LANG WHERE FIRST_APPEARED=1972", con)

con.close()

df


[('C', 1972)]


,NAME,FIRST_APPEARED
0,C,1972
